In [162]:
import great_expectations as ge
import pandas as pd

In [163]:
context_seq = ge.data_context.DataContext()

In [164]:
suite = context_seq.create_expectation_suite(
    'check_seq_data',
    overwrite_existing=True
)

In [165]:
context_seq.add_datasource("tsv", class_name='PandasDatasource')

In [173]:
batch_kwargs = {
    'path': 'tsv/seq_stats.tsv',
    'datasource': 'tsv',
    'data_asset_name': 'seq_stats',
    'reader_method': 'read_csv',
    'reader_options': {
        'sep': '\t'
    }
}
batch = context_seq.get_batch(batch_kwargs, suite)

In [167]:
batch.head()

,n_contigs,contig_bp,gap_pct,ctg_N50,ctg_L50,ctg_N90,ctg_L90,ctg_max,gc_avg,gc_std,filename
0,100,11670,0.000,34,130,83,74,265,0.47215,0.12871,/home/huangsixing/Documents/mgrast_ge/fasta/mg...
1,100,22815,0.018,25,420,69,84,512,0.51269,0.10778,/home/huangsixing/Documents/mgrast_ge/fasta/mg...
2,100,22025,0.005,26,375,74,91,495,0.54561,0.10954,/home/huangsixing/Documents/mgrast_ge/fasta/mg...
3,100,23148,0.009,26,378,70,99,503,0.53599,0.09698,/home/huangsixing/Documents/mgrast_ge/fasta/mg...
4,100,12116,0.000,30,139,81,67,493,0.48382,0.10773,/home/huangsixing/Documents/mgrast_ge/fasta/mg...


In [128]:
batch.expect_column_values_to_be_between('ctg_L50', min_value=80, max_value=600, result_format={'result_format': 'COMPLETE'})

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 49,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": [],
    "partial_unexpected_index_list": [],
    "partial_unexpected_counts": [],
    "unexpected_list": [],
    "unexpected_index_list": []
  },
  "success": true,
  "meta": {}
}

In [129]:
batch.expect_column_values_to_be_between('n_contigs', min_value=1, max_value=100, result_format={'result_format': 'COMPLETE'})

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 49,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": [],
    "partial_unexpected_index_list": [],
    "partial_unexpected_counts": [],
    "unexpected_list": [],
    "unexpected_index_list": []
  },
  "success": true,
  "meta": {}
}

In [130]:
batch.expect_column_values_to_be_between('gc_avg', min_value=0.4, max_value=0.6, result_format={'result_format': 'COMPLETE'})

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 49,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": [],
    "partial_unexpected_index_list": [],
    "partial_unexpected_counts": [],
    "unexpected_list": [],
    "unexpected_index_list": []
  },
  "success": true,
  "meta": {}
}

In [131]:
seq_expectation_suite = batch.get_expectation_suite(discard_failed_expectations=False)

In [132]:
batch.save_expectation_suite()

In [133]:
validation_operator_name = 'seq_validation_operator'
validation_operator_config = {
    'class_name': 'ActionListValidationOperator',
    'action_list': [{'name': 'store_validation_result', 'action': {'class_name': 'StoreValidationResultAction'}}, {'name': 'update_data_docs', 'action': {'class_name': 'UpdateDataDocsAction'}}]
}
context_seq.add_validation_operator(validation_operator_name, validation_operator_config)

In [134]:
results = context_seq.run_validation_operator('seq_validation_operator', assets_to_validate=[batch])

In [112]:
results

{
  "validation_operator_config": {
    "class_name": "ActionListValidationOperator",
    "module_name": "great_expectations.validation_operators",
    "name": "seq_validation_operator",
    "kwargs": {
      "action_list": [
        {
          "name": "store_validation_result",
          "action": {
            "class_name": "StoreValidationResultAction"
          }
        },
        {
          "name": "update_data_docs",
          "action": {
            "class_name": "UpdateDataDocsAction"
          }
        }
      ],
      "result_format": {
        "result_format": "SUMMARY",
        "partial_unexpected_count": 20
      }
    }
  },
  "run_id": {
    "run_name": "20220531T173220.488202Z",
    "run_time": "2022-05-31T17:32:20.488202+00:00"
  },
  "success": true,
  "evaluation_parameters": null,
  "run_results": {
    "ValidationResultIdentifier::check_seq_data/20220531T173220.488202Z/20220531T173220.488202Z/e26b49f837c84090cd19c58a1ae22a29": {
      "validation_result": {
   

In [135]:
context_seq.open_data_docs()

In [136]:
%%writefile great_expectations/checkpoints/test_seq_data.yml

validation_operator_name: seq_validation_operator
batches:
  - batch_kwargs:
      path: tsv/test_seq_stats.tsv
      datasource: tsv
      data_asset_name: test_seq_stats
      reader_method: read_csv
      reader_options:
        'sep': '\t'
    expectation_suite_names:
      - check_seq_data

Writing great_expectations/checkpoints/test_seq_data.yml


In [137]:
!great_expectations checkpoint run test_seq_data

Using v3 (Batch Request) API
Validation failed!

Suite Name                                   Status     Expectations met
- check_seq_data                             ✖ Failed   1 of 3 (33.33 %)


Checkpoint store named "checkpoint_store" is not a configured store, so will try to use default Checkpoint store.
  Please update your configuration to the new version number 3.0 in order to use the new "Checkpoint Store" feature.
  Visit https://docs.greatexpectations.io/docs/guides/miscellaneous/migration_guide#migrating-to-the-batch-request-v3-api to learn more about the upgrade process.


In [138]:
test_seq_df = pd.read_csv('tsv/test_seq_stats.tsv', sep='\t')
ge_df = ge.from_pandas(test_seq_df)

In [139]:
result = ge_df.validate(seq_expectation_suite, result_format="COMPLETE")

In [140]:
#ge_df.iloc[result.results[0].result["unexpected_index_list"]]

for i in range(len(result.results)):
    r = result.results[i]
    if r["success"] == False:
        column = r.expectation_config["kwargs"]["column"]
        expectation_type = r.expectation_config["expectation_type"]
        print (f"{column} {expectation_type} failed")
        print (ge_df.iloc[r.result["unexpected_index_list"]])
        print ("\n")

ctg_L50 expect_column_values_to_be_between failed
   n_contigs  contig_bp  gap_pct  ctg_N50  ctg_L50  ctg_N90  ctg_L90  ctg_max  \
0         31       2775      0.0       30       60       30       90       90   

    gc_avg   gc_std                                           filename  
0  0.55423  0.02767  /home/huangsixing/Documents/mgrast_ge/test_fas...  


gc_avg expect_column_values_to_be_between failed
   n_contigs  contig_bp  gap_pct  ctg_N50  ctg_L50  ctg_N90  ctg_L90  ctg_max  \
1         69       9257    0.032       23      149       55       85      363   

    gc_avg   gc_std                                           filename  
1  0.31323  0.09431  /home/huangsixing/Documents/mgrast_ge/test_fas...  




In [141]:
result

{
  "evaluation_parameters": {},
  "success": false,
  "results": [
    {
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      },
      "result": {
        "element_count": 2,
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_count": 1,
        "unexpected_percent": 50.0,
        "unexpected_percent_total": 50.0,
        "unexpected_percent_nonmissing": 50.0,
        "partial_unexpected_list": [
          60
        ],
        "partial_unexpected_index_list": [
          0
        ],
        "partial_unexpected_counts": [
          {
            "value": 60,
            "count": 1
          }
        ],
        "unexpected_list": [
          60
        ],
        "unexpected_index_list": [
          0
        ]
      },
      "expectation_config": {
        "kwargs": {
          "column": "ctg_L50",
          "min_value": 80,
          "max_value": 600,
          "result_f

In [182]:
test_seq_df = pd.read_csv('tsv/test_seq_stats.tsv', sep='\t')
ge_df = ge.from_pandas(test_seq_df)

ge.profile.BasicSuiteBuilderProfiler().profile(ge_df)

Profiling Columns:   0%|          | 0/11 [00:00<?, ?it/s, n_contigs]

c:\Users\dgg32\.conda\envs\great_expectations\lib\site-packages\great_expectations\data_asset\data_asset.py:458: DeprecationWarning: DataAsset.remove_expectations is deprecated as of v0.12.0 and will be removed in v0.16. Please use ExpectationSuite.remove_expectation instead.
  warnings.warn(
c:\Users\dgg32\.conda\envs\great_expectations\lib\site-packages\great_expectations\data_asset\data_asset.py:458: DeprecationWarning: DataAsset.remove_expectations is deprecated as of v0.12.0 and will be removed in v0.16. Please use ExpectationSuite.remove_expectation instead.
  warnings.warn(
c:\Users\dgg32\.conda\envs\great_expectations\lib\site-packages\great_expectations\data_asset\data_asset.py:458: DeprecationWarning: DataAsset.remove_expectations is deprecated as of v0.12.0 and will be removed in v0.16. Please use ExpectationSuite.remove_expectation instead.
  warnings.warn(
c:\Users\dgg32\.conda\envs\great_expectations\lib\site-packages\great_expectations\data_asset\data_asset.py:458: Depre

({
   "data_asset_type": "Dataset",
   "expectation_suite_name": "default",
   "meta": {
     "great_expectations_version": "0.15.7",
     "columns": {
       "n_contigs": {
         "description": ""
       },
       "contig_bp": {
         "description": ""
       },
       "gap_pct": {
         "description": ""
       },
       "ctg_N50": {
         "description": ""
       },
       "ctg_L50": {
         "description": ""
       },
       "ctg_N90": {
         "description": ""
       },
       "ctg_L90": {
         "description": ""
       },
       "ctg_max": {
         "description": ""
       },
       "gc_avg": {
         "description": ""
       },
       "gc_std": {
         "description": ""
       },
       "filename": {
         "description": ""
       }
     },
     "BasicSuiteBuilderProfiler": {
       "created_by": "BasicSuiteBuilderProfiler",
       "created_at": 1654021392.6901891,
       "batch_kwargs": {
         "ge_batch_id": "bb062940-e10e-11ec-9cd6-54b2031697